In [3]:
%load_ext autoreload
%autoreload 2

import os
import sys
sys.path.append(os.path.dirname(os.path.abspath(os.path.join('../'))))

import tqdm
import numpy
import torch
import wandb
import pandas
import joblib
import itertools
import torchvision
import gtda.images
import gtda.diagrams
import gtda.homology
import sklearn.pipeline
import sklearn.ensemble
import sklearn.metrics
import tqdm.contrib.itertools
import sklearn.decomposition

import lib.topology

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
BINARIZATION_THESHOLD = 0.4

height_filtration_directions = [
    [ -1, -1 ], [ 1, 1 ], [ 1, -1 ], [ -1, 1 ],
    [ 0, -1 ], [ 0, 1 ], [ -1, 0 ], [ 1, 0 ]
]


radial_filtration_centers = list(itertools.product([ 7, 14, 21 ], [ 7, 14, 21 ]))
radial_filtration_metrics = [ "euclidean", "manhattan", "cosine" ]

density_filtration_metrics = [ "euclidean" , "manhattan", "cosine" ]
density_filtration_radiuses = [ 1, 5, 15 ]

FILTRATIONS = [
    *[ [ gtda.images.HeightFiltration, { 'direction': numpy.array(direction), 'n_jobs': -1 } ] for direction in height_filtration_directions ],
    *[
        [ gtda.images.RadialFiltration, { 'center': numpy.array(center), 'metric': metric, 'n_jobs': -1 } ]
        for center in radial_filtration_centers
        for metric in radial_filtration_metrics
    ],
    [ gtda.images.DilationFiltration, { 'n_jobs': -1 } ],
    [ gtda.images.ErosionFiltration, { 'n_jobs': -1 } ],
    [ gtda.images.SignedDistanceFiltration, { 'n_jobs': -1 } ],
    *[
        [ gtda.images.DensityFiltration, { 'radius': radius, 'metric': metric, 'n_jobs': -1 } ]
        for metric in density_filtration_metrics
        for radius in density_filtration_radiuses
    ]
]

In [5]:
train = torchvision.datasets.MNIST('mnist', train = True, download = True)
test = torchvision.datasets.MNIST('mnist', train = False, download = True)

train_images = numpy.array([ item[0] for item in train ])
train_labels = numpy.array([ item[1] for item in train ])

test_images = numpy.array([ item[0] for item in test ])
test_labels = numpy.array([ item[1] for item in test ])

In [6]:
def make_filtrations(images: numpy.ndarray):
    images_bin = gtda.images.Binarizer(threshold = BINARIZATION_THESHOLD).fit_transform(images)
    filtrations = [
        filtration[0](**filtration[1]).fit_transform(images_bin)
        for filtration in tqdm.tqdm(FILTRATIONS, desc = 'filtrations')
    ]
    return [ images, images_bin ] + filtrations


def make_point_clouds(images: numpy.ndarray):
    def make_point_cloud(image, threshold):
        point_cloud = [ ]
        for i in range(image.shape[0]):
            for j in range(image.shape[1]):
                if image[i][j] < threshold:
                    continue
                point_cloud.append([ i, j, image[i][j] ])
        return numpy.array(point_cloud)

    def _make_point_clouds(imgs, threshold):
        imgs = numpy.swapaxes(numpy.flip(imgs, axis = 1), 1, 2)
        return [ make_point_cloud(image, threshold) for image in tqdm.tqdm(imgs, desc = 'point_clouds') ]

    images_bin = gtda.images.Binarizer(threshold = BINARIZATION_THESHOLD).fit_transform(images)
    point_cloud = gtda.images.ImageToPointCloud().fit_transform(images_bin)
    return [ point_cloud, _make_point_clouds(images, 50) ]

In [7]:
def make_filtration_diagrams(images: numpy.ndarray):
    filtrations = make_filtrations(images)
    all_filtrations = [ ]
    for diagrams in zip(*filtrations):
        all_filtrations.extend(diagrams)

    print('Making filtration diagrams')
    persistence = gtda.homology.CubicalPersistence(homology_dimensions = [ 0, 1 ], n_jobs = -1)
    return persistence.fit_transform(all_filtrations)

def make_point_cloud_diagrams(images: numpy.ndarray):
    point_clouds = make_point_clouds(images)
    all_point_clouds = [ ]
    for diagrams in zip(*point_clouds):
        all_point_clouds.extend(diagrams)
        
    print('Making point cloud diagrams')
    persistence = gtda.homology.VietorisRipsPersistence(homology_dimensions = [ 0, 1, 2 ], n_jobs = -1)
    return persistence.fit_transform(all_point_clouds)

In [8]:
def make_filtration_features(images: numpy.ndarray):
    feature_calculator = lib.topology.FeatureCalculator(n_jobs = -1, verbose = True)
    return feature_calculator.calc_features(make_filtration_diagrams(images), prefix = 'filtration')

def make_point_cloud_features(images: numpy.ndarray):
    feature_calculator = lib.topology.FeatureCalculator(n_jobs = -1, verbose = True)
    return feature_calculator.calc_features(make_point_cloud_diagrams(images), prefix = 'point_cloud')

def make_features(images: numpy.ndarray):
    filtration_features = make_filtration_features(images)
    point_cloud_features = make_point_cloud_features(images)

    filtrations_per_image = len(filtration_features) // len(images)
    point_clouds_per_image = len(point_cloud_features) // len(images)
    
    features = [ ]
    for img_idx in tqdm.trange(len(images), desc = 'combine features'):
        idx1 = filtrations_per_image * img_idx
        idx2 = filtrations_per_image * (img_idx + 1)
        filtration_features_flat = filtration_features[idx1:idx2].to_numpy().flatten()
        
        idx1 = point_clouds_per_image * img_idx
        idx2 = point_clouds_per_image * (img_idx + 1)
        point_cloud_features_flat = point_cloud_features[idx1:idx2].to_numpy().flatten()
        
        features_flat = numpy.concatenate((filtration_features_flat, point_cloud_features_flat), axis = 0)
        features.append(features_flat)

    return numpy.array(features)

In [7]:
train_features = None
for i in range(0, 60000, 4000):
    print(f'Calculating {i} - {i + 4000}')
    train_features_batch = make_features(train_images[i:i + 4000])
    if train_features is None:
        train_features = train_features_batch
    else:
        train_features = numpy.concatenate((train_features, train_features_batch), axis = 0)
    numpy.save("train_features.npy", train_features)

Calculating 0 - 4000


filtrations: 100%|██████████| 47/47 [00:15<00:00,  3.00it/s]


Making filtration diagrams
Filtered diagrams: (196000, 71, 3)
Calculating Betti features


filtration betti: 100%|██████████| 196000/196000 [01:04<00:00, 3059.93it/s]


Calculating landscape features


...

point_cloud silhouette-2: 100%|██████████| 8000/8000 [00:03<00:00, 2100.88it/s]


Calculating entropy features
Calculating number of points features
Calculating amplitude features


point_cloud amplitudes: 100%|██████████| 17/17 [00:18<00:00,  1.08s/it]


Calculating lifetime features


combine features: 100%|██████████| 4000/4000 [00:01<00:00, 2660.91it/s]


In [11]:
test_features = None
for i in range(0, 10000, 2500):
    print(f'Calculating {i} - {i + 2500}')
    test_features_batch = make_features(test_images[i:i + 2500])
    if test_features is None:
        test_features = test_features_batch
    else:
        test_features = numpy.concatenate((test_features, test_features_batch), axis = 0)
    numpy.save("test_features.npy", test_features)

Calculating 0 - 2500


filtrations: 100%|██████████| 47/47 [00:08<00:00,  5.73it/s]


Making filtration diagrams
Filtered diagrams: (122500, 56, 3)
Calculating Betti features


filtration betti: 100%|██████████| 122500/122500 [00:42<00:00, 2869.31it/s]


Calculating landscape features


...

point_cloud silhouette-2: 100%|██████████| 5000/5000 [00:02<00:00, 1857.95it/s]


Calculating entropy features
Calculating number of points features
Calculating amplitude features


point_cloud amplitudes: 100%|██████████| 17/17 [00:12<00:00,  1.32it/s]


Calculating lifetime features


combine features: 100%|██████████| 2500/2500 [00:00<00:00, 2540.88it/s]


In [9]:
train_features = numpy.load("train_features.npy")
test_features = numpy.load("test_features.npy")

In [4]:
train_features = numpy.minimum(train_features, 1e9)
test_features = numpy.minimum(test_features, 1e9)

In [5]:
rf = sklearn.ensemble.RandomForestClassifier(n_jobs = -1, n_estimators = 1000)
rf.fit(train_features, train_labels)
print(rf.score(test_features, test_labels))

0.9736


In [8]:
pca = sklearn.decomposition.PCA(n_components = 8, random_state = 42)
train_features_reduced = pca.fit_transform(train_features)
print('Explained variance', round(pca.explained_variance_ratio_.sum(), 2))
print([ round(x, 3) for x in pca.explained_variance_ratio_ ])
test_features_reduced = pca.transform(test_features)

Explained variance 1.0
[0.476, 0.381, 0.143, 0.0, 0.0, 0.0, 0.0, 0.0]


In [ ]:
rf_reduced = sklearn.ensemble.RandomForestClassifier(n_jobs = -1, n_estimators = 10000)
rf_reduced.fit(train_features_reduced, train_labels)
print(rf_reduced.score(test_features_reduced, test_labels))